### Import external python files

In [55]:
%run skeleton.py

### Load the skeleton definition
This is pickled from maya

In [56]:
import pickle
base_skeleton = pickle.load(open('skeleton_no_fingers.dat','rb'))
base_skeleton.convert_to_np()

### Draw Skeleton
We will use k3d to display the skeleton in a view port

In [80]:
import k3d

#capsule
capsule_vertices =  [[-1.9238795042037964, -0.2705979347229004, -0.27059805393218994], [-1.923879623413086, 1.1920928955078125e-07, -0.3826833963394165], [-1.9238793849945068, 0.27059805393218994, -0.27059805393218994], [-1.9238795042037964, 0.38268351554870605, 0.0], [-1.9238793849945068, 0.27059805393218994, 0.27059805393218994], [-1.923879623413086, 1.1920928955078125e-07, 0.3826834261417389], [-1.9238795042037964, -0.2705979347229004, 0.27059805393218994], [-1.9238793849945068, -0.38268333673477173, 0.0], [-1.707106590270996, -0.49999985098838806, -0.4999999403953552], [-1.7071067094802856, 1.7881393432617188e-07, -0.7071067094802856], [-1.7071068286895752, 0.5000000596046448, -0.4999999403953552], [-1.7071067094802856, 0.7071067690849304, 0.0], [-1.7071068286895752, 0.5000000596046448, 0.4999999403953552], [-1.7071067094802856, 1.7881393432617188e-07, 0.7071067094802856], [-1.707106590270996, -0.49999985098838806, 0.4999999701976776], [-1.7071068286895752, -0.7071066498756409, 0.0], [-1.382683277130127, -0.6532813310623169, -0.6532813906669617], [-1.3826833963394165, 5.960464477539063e-08, -0.9238793849945068], [-1.3826833963394165, 0.6532814502716064, -0.6532813906669617], [-1.382683277130127, 0.9238793849945068, 0.0], [-1.3826833963394165, 0.6532814502716064, 0.6532813906669617], [-1.3826833963394165, 5.960464477539063e-08, 0.9238794445991516], [-1.382683515548706, -0.6532814502716064, 0.6532814502716064], [-1.3826833963394165, -0.9238795042037964, 0.0], [1.382683515548706, -0.6532813310623169, -0.6532813906669617], [1.3826833963394165, 8.940696716308594e-08, -0.9238793849945068], [1.382683515548706, 0.6532814502716064, -0.6532813906669617], [1.382683515548706, 0.9238795042037964, 0.0], [1.382683515548706, 0.6532814502716064, 0.6532813906669617], [1.3826833963394165, 8.940696716308594e-08, 0.9238794445991516], [1.3826833963394165, -0.6532814502716064, 0.6532814502716064], [1.382683515548706, -0.9238795042037964, 0.0], [1.7071068286895752, -0.4999998211860657, -0.4999999403953552], [1.7071067094802856, 5.960464477539063e-08, -0.7071067094802856], [1.707106590270996, 0.5000001192092896, -0.4999999403953552], [1.7071069478988647, 0.7071067094802856, 0.0], [1.707106590270996, 0.5000001192092896, 0.4999999403953552], [1.7071067094802856, 5.960464477539063e-08, 0.7071067094802856], [1.7071069478988647, -0.4999999403953552, 0.4999999701976776], [1.7071068286895752, -0.7071067690849304, 0.0], [1.9238795042037964, -0.2705979347229004, -0.27059805393218994], [1.923879623413086, 5.960464477539063e-08, -0.3826833963394165], [1.9238795042037964, 0.27059799432754517, -0.27059805393218994], [1.9238795042037964, 0.38268333673477173, 0.0], [1.9238795042037964, 0.27059799432754517, 0.27059805393218994], [1.923879623413086, 5.960464477539063e-08, 0.3826834261417389], [1.9238795042037964, -0.2705979347229004, 0.27059805393218994], [1.923879623413086, -0.38268351554870605, 0.0], [-2.0, 0.0, 0.0], [2.0, 0.0, 0.0], [-1.007652759552002, -0.7060302495956421, -0.7060303092002869], [5.960464477539063e-08, -0.7065632939338684, -0.7065632939338684], [1.007652997970581, -0.7060302495956421, -0.7060303092002869], [-1.007652759552002, 6.51925802230835e-08, -0.9984776377677917], [5.960464477539063e-08, 5.960464477539063e-08, -0.9992313981056213], [1.007652997970581, 8.381903171539307e-08, -0.9984776377677917], [-1.0076528787612915, 0.7060303688049316, -0.7060303092002869], [5.960464477539063e-08, 0.7065633535385132, -0.7065632939338684], [1.007652759552002, 0.7060303092002869, -0.7060303092002869], [-1.007652759552002, 0.998477578163147, 0.0], [5.960464477539063e-08, 0.9992314577102661, 0.0], [1.007652759552002, 0.9984776973724365, 0.0], [-1.0076528787612915, 0.7060303688049316, 0.7060303092002869], [5.960464477539063e-08, 0.7065633535385132, 0.7065632939338684], [1.007652759552002, 0.7060303092002869, 0.7060303092002869], [-1.007652759552002, 6.51925802230835e-08, 0.9984776973724365], [5.960464477539063e-08, 5.960464477539063e-08, 0.9992314577102661], [1.007652997970581, 8.381903171539307e-08, 0.9984776973724365], [-1.007652759552002, -0.7060302495956421, 0.7060303688049316], [5.960464477539063e-08, -0.7065632939338684, 0.7065633535385132], [1.007652997970581, -0.7060303092002869, 0.7060303688049316], [-1.007652759552002, -0.9984776973724365, 0.0], [5.960464477539063e-08, -0.9992314577102661, 0.0], [1.0076528787612915, -0.9984776973724365, 0.0]]
capsule_indices =  [[0, 1, 9], [1, 2, 10], [2, 3, 11], [3, 4, 12], [4, 5, 13], [5, 6, 14], [6, 7, 15], [7, 0, 8], [8, 9, 17], [9, 10, 18], [10, 11, 19], [11, 12, 20], [12, 13, 21], [13, 14, 22], [14, 15, 23], [15, 8, 16], [24, 25, 33], [25, 26, 34], [26, 27, 35], [27, 28, 36], [28, 29, 37], [29, 30, 38], [30, 31, 39], [31, 24, 32], [32, 33, 41], [33, 34, 42], [34, 35, 43], [35, 36, 44], [36, 37, 45], [37, 38, 46], [38, 39, 47], [39, 32, 40], [1, 0, 48], [2, 1, 48], [3, 2, 48], [4, 3, 48], [5, 4, 48], [6, 5, 48], [7, 6, 48], [0, 7, 48], [40, 41, 49], [41, 42, 49], [42, 43, 49], [43, 44, 49], [44, 45, 49], [45, 46, 49], [46, 47, 49], [47, 40, 49], [16, 17, 53], [17, 18, 56], [18, 19, 59], [19, 20, 62], [20, 21, 65], [21, 22, 68], [22, 23, 71], [23, 16, 50], [52, 55, 25], [55, 58, 26], [58, 61, 27], [61, 64, 28], [64, 67, 29], [67, 70, 30], [70, 73, 31], [73, 52, 24], [50, 51, 72], [51, 52, 73], [52, 51, 54], [51, 50, 53], [55, 54, 57], [54, 53, 56], [58, 57, 60], [57, 56, 59], [61, 60, 63], [60, 59, 62], [64, 63, 66], [63, 62, 65], [67, 66, 69], [66, 65, 68], [70, 69, 72], [69, 68, 71]]
capsule_v = [i for v in capsule_vertices for i in v]
capsule_i = [i for v in capsule_indices for i in v]

plot = k3d.plot()

for i in range(len(base_skeleton._bones)):
    matrix = base_skeleton.globalMatrix(i)
    mesh = k3d.mesh(capsule_v, capsule_i)
    mesh.model_matrix = matrix.T
    plot += mesh
plot.display()


Output()